# Classification. Decision Trees & k - Nearest Neighbors

Со второй части лекции ЮК. На оттоке клиентов телеком компании

In [1]:
# we don't like warnings
# you can comment the following 2 lines if you'd like to
import warnings

warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import seaborn as sns

sns.set()
from matplotlib import pyplot as plt

%config InlineBackend.figure_format = 'retina'

In [28]:
df = pd.read_csv("telecom_churn.csv")

In [29]:
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


### Предобработка

In [30]:
df.drop(['state', 'phone number', 'voice mail plan'], axis=1, inplace=True) #Уберем лишнии не числовые

In [31]:
df['international plan'].value_counts()

no     3010
yes     323
Name: international plan, dtype: int64

In [32]:
df['international plan'] = df['international plan'].map({'no': 0, 'yes': 1}) #Из да/нет сделаем числовой 1/0

Если есть значения не равные да или нет, то в sklearn есть какие-то модули (какие не сказал)

In [33]:
df['churn'] = df['churn'].astype('int64') #Из boolсделаем числовой

In [34]:
df.info() #Проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   account length          3333 non-null   int64  
 1   area code               3333 non-null   int64  
 2   international plan      3333 non-null   int64  
 3   number vmail messages   3333 non-null   int64  
 4   total day minutes       3333 non-null   float64
 5   total day calls         3333 non-null   int64  
 6   total day charge        3333 non-null   float64
 7   total eve minutes       3333 non-null   float64
 8   total eve calls         3333 non-null   int64  
 9   total eve charge        3333 non-null   float64
 10  total night minutes     3333 non-null   float64
 11  total night calls       3333 non-null   int64  
 12  total night charge      3333 non-null   float64
 13  total intl minutes      3333 non-null   float64
 14  total intl calls        3333 non-null   

In [36]:
X = df.drop(['churn'], axis=1) #Конечный фрейм

In [39]:
df.shape, X.shape

((3333, 18), (3333, 17))

In [41]:
y = df['churn'] #Целевая переменная

X и Y готовы, можно приступать.

### Разбиваем на трен/валид (здесь в соотношении 70 на 30)

In [47]:
#Сначала разобъем в соотношении 70 на 30. Тренируем на 70, проверяем на 30% выборке из нашей выборки.
from sklearn.model_selection import train_test_split

In [48]:
#random_state - перемешивание перед разбивкой. Лучше фиксировать, и не оставлять пустым.
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=17)

In [52]:
X_train.shape, X_test.shape #Просто убедимся, что 70 на 30 %, как заказывали

((2333, 17), (1000, 17))

## Обучаем

### Dicision Trees

In [53]:
from sklearn.tree import DecisionTreeClassifier
#Создаем экземпляр класса (конструктор), лучше также фиксировать random_state
f_tree = DecisionTreeClassifier(random_state=17)

In [54]:
#Собственно, само обучение
f_tree_train = f_tree.fit(X_train, y_train)

In [57]:
#Считаем какой % правильных ответов будет при применении деревьев на тренировочной выборке
#при кросс-валидации на 5-ти фолдах
from sklearn.model_selection import cross_val_score

In [60]:
cross_val_score(f_tree, X_train, y_train, cv=5) #Так получим по значению на каждом фолде (у нас - 5)
#Здесь надо на эти цифры посмотреть. Они не должны иметь огромные различия. Обычно так и есть.

array([0.9143469 , 0.91220557, 0.92077088, 0.90772532, 0.91416309])

In [62]:
round(np.mean(cross_val_score(f_tree, X_train, y_train, cv=5)), 4) #Вычислим среднее и округлим

0.9138

In [63]:
round(cross_val_score(f_tree, X_train, y_train, cv=5).mean(), 4) #Можно и так

0.9138

In [64]:
round(np.median(cross_val_score(f_tree, X_train, y_train, cv=5)), 4) #Можно медиану проверить на всяк случай

0.9142

### k-Nearest Neighbors

In [67]:
from sklearn.neighbors import KNeighborsClassifier

In [68]:
f_knn = KNeighborsClassifier()

In [70]:
KNeighborsClassifier?

In [69]:
cross_val_score(f_knn, X_train, y_train, cv=5) #Смотрим кросс-валидацию также как и у деревьев

array([0.8608137 , 0.85653105, 0.875803  , 0.86266094, 0.87982833])

In [72]:
round(cross_val_score(f_knn, X_train, y_train, cv=5).mean(), 4) #Значительно похуже, чем у деревьев!

0.8671

In [76]:
round(1 - y.mean(), 4) #А такая вероятность получения 1, если мы поставим всем 1 в прогноз. Т.е. без обучения

0.8551

## Подберем наилучшие гиперпараметры!

Обязательно, т.к. возможно изменение значений у всех моделей!

### 1.Максимальная глубина дерева решений - max_depth

!!!При большой решетке очень энергозатратно!!!

In [78]:
#Поисковик наилучших гиперпараметров!!! "Решетчатый поиск кросс-вал"
from sklearn.model_selection import GridSearchCV
#Решетка, т.к. поиск по всем сочетаниям указанных гиперпараметров в словарях или массивах

In [103]:
d_param = {'max_depth': np.arange(1, 11, 1), 'max_features': np.arange(0.3, 1, 0.05)}
#Так и будет перебор по решетке из 30 точек (10 первых параметров по 3-м значениям второго параметра)

In [104]:
#n_jobs - кол-во потоков, которые задействует, так - все, остальное затормооозииитттсссяяяяя))))
tree = GridSearchCV(f_tree, param_grid= d_param, cv=5, n_jobs=-1, verbose=3)

In [121]:
%%time
tree_grid = tree.fit(X_train, y_train); #Вот он обученный объект на трейн выборке!!!!!!!!!

Fitting 5 folds for each of 140 candidates, totalling 700 fits
[CV 1/5] END .....max_depth=1, max_features=0.3;, score=0.837 total time=   0.0s
[CV 5/5] END .....max_depth=1, max_features=0.3;, score=0.873 total time=   0.0s
[CV 1/5] END ....max_depth=1, max_features=0.35;, score=0.837 total time=   0.0s
[CV 3/5] END ....max_depth=1, max_features=0.35;, score=0.876 total time=   0.0s
[CV 5/5] END ....max_depth=1, max_features=0.35;, score=0.873 total time=   0.0s
[CV 3/5] END max_depth=1, max_features=0.39999999999999997;, score=0.876 total time=   0.0s
[CV 1/5] END max_depth=1, max_features=0.44999999999999996;, score=0.837 total time=   0.0s
[CV 4/5] END max_depth=1, max_features=0.44999999999999996;, score=0.863 total time=   0.0s
[CV 2/5] END max_depth=1, max_features=0.49999999999999994;, score=0.869 total time=   0.0s
[CV 1/5] END max_depth=1, max_features=0.5499999999999999;, score=0.837 total time=   0.0s
[CV 5/5] END max_depth=1, max_features=0.5499999999999999;, score=0.873 t

[CV 4/5] END .....max_depth=1, max_features=0.3;, score=0.863 total time=   0.0s
[CV 3/5] END max_depth=1, max_features=0.49999999999999994;, score=0.876 total time=   0.0s
[CV 2/5] END max_depth=1, max_features=0.5499999999999999;, score=0.869 total time=   0.0s
[CV 1/5] END max_depth=1, max_features=0.5999999999999999;, score=0.837 total time=   0.0s
[CV 2/5] END max_depth=1, max_features=0.6499999999999999;, score=0.869 total time=   0.0s
[CV 3/5] END max_depth=1, max_features=0.6499999999999999;, score=0.876 total time=   0.0s
[CV 5/5] END .....max_depth=1, max_features=0.7;, score=0.873 total time=   0.0s
[CV 1/5] END max_depth=1, max_features=0.7499999999999999;, score=0.837 total time=   0.0s
[CV 1/5] END max_depth=1, max_features=0.8999999999999999;, score=0.837 total time=   0.0s
[CV 2/5] END max_depth=1, max_features=0.8999999999999999;, score=0.869 total time=   0.0s
[CV 3/5] END max_depth=1, max_features=0.8999999999999999;, score=0.876 total time=   0.0s
[CV 4/5] END max_d

[CV 3/5] END .....max_depth=1, max_features=0.3;, score=0.876 total time=   0.0s
[CV 4/5] END max_depth=1, max_features=0.39999999999999997;, score=0.863 total time=   0.0s
[CV 2/5] END max_depth=1, max_features=0.44999999999999996;, score=0.869 total time=   0.0s
[CV 5/5] END max_depth=1, max_features=0.44999999999999996;, score=0.873 total time=   0.0s
[CV 5/5] END max_depth=1, max_features=0.49999999999999994;, score=0.873 total time=   0.0s
[CV 4/5] END max_depth=1, max_features=0.5499999999999999;, score=0.863 total time=   0.0s
[CV 5/5] END max_depth=1, max_features=0.5999999999999999;, score=0.873 total time=   0.0s
[CV 1/5] END max_depth=1, max_features=0.6499999999999999;, score=0.837 total time=   0.0s
[CV 3/5] END .....max_depth=1, max_features=0.7;, score=0.876 total time=   0.0s
[CV 4/5] END .....max_depth=1, max_features=0.7;, score=0.863 total time=   0.0s
[CV 3/5] END max_depth=1, max_features=0.7999999999999998;, score=0.876 total time=   0.0s
[CV 4/5] END max_depth=1,

CPU times: user 904 ms, sys: 111 ms, total: 1.01 s
Wall time: 7.99 s


In [122]:
#Вот они - лучший процент правильных ответов, лучшие параметры, т.е. лучшая модель
round(tree_greed.best_score_, 4), tree_greed.best_params_
#94% - круууче, чем 91 или 92, которые были не на лучших параметрах!

(0.9417, {'max_depth': 6, 'max_features': 0.8999999999999999})

### Наилучшее количество соседей "к"

   Не забываем заимпортить GridSearchCV 

In [130]:
k_param = {'n_neighbors': np.arange(10, 50, 5)}

In [131]:
knn = GridSearchCV(f_knn, k_param, cv=5, n_jobs=-1, verbose=3)

In [132]:
%%time
knn_grid = knn.fit(X_tarin, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 3/5] END ....max_depth=7, max_features=0.35;, score=0.912 total time=   0.0s
[CV 4/5] END ....max_depth=7, max_features=0.35;, score=0.927 total time=   0.0s
[CV 5/5] END ....max_depth=7, max_features=0.35;, score=0.891 total time=   0.0s
[CV 1/5] END max_depth=7, max_features=0.39999999999999997;, score=0.906 total time=   0.0s
[CV 2/5] END max_depth=7, max_features=0.39999999999999997;, score=0.914 total time=   0.0s
[CV 1/5] END max_depth=7, max_features=0.8999999999999999;, score=0.936 total time=   0.0s
[CV 2/5] END max_depth=7, max_features=0.8999999999999999;, score=0.938 total time=   0.0s
[CV 3/5] END max_depth=7, max_features=0.8999999999999999;, score=0.936 total time=   0.1s
[CV 4/5] END max_depth=7, max_features=0.8999999999999999;, score=0.940 total time=   0.1s
[CV 5/5] END max_depth=7, max_features=0.8999999999999999;, score=0.938 total time=   0.0s
[CV 1/5] END ....max_depth=7, max_features=0.95;, score=0.

CPU times: user 118 ms, sys: 13 ms, total: 131 ms
Wall time: 1.14 s


In [134]:
round(knn_grid.best_score_, 4),  knn_grid.best_params_ #kNN мягко скажем, не очень здесь работает

(0.8701, {'n_neighbors': 10})

[CV 2/5] END .....................n_neighbors=8;, score=0.872 total time=   0.1s
[CV 3/5] END .....................n_neighbors=8;, score=0.872 total time=   0.1s
[CV 5/5] END .....................n_neighbors=9;, score=0.882 total time=   0.1s
[CV 1/5] END ....................n_neighbors=10;, score=0.859 total time=   0.1s
[CV 4/5] END ....................n_neighbors=10;, score=0.873 total time=   0.1s
[CV 4/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 2/5] END .....................n_neighbors=2;, score=0.865 total time=   0.1s
[CV 2/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 3/5] END .......................n_neighbors=0;, score=nan total time=   0.0s
[CV 3/5] END .....................n_neighbors=4;, score=0.869 total time=   0.1s
[CV 4/5] END .....................n_neighbors=4;, score=0.869 total time=   0.1s
[CV 1/5] END .....................n_neighbors=8;, score=0.854 total time=   0.1s
[CV 4/5] END ...............

Все понятно, как не ищи, но здесь деревья лучше соседей работают!

### Смотрим как будет работать обученная лучшая модель на тестовой выборке

In [135]:
#Лучшая модель
tree_grid.best_estimator_

DecisionTreeClassifier(max_depth=6, max_features=0.8999999999999999,
                       random_state=17)

In [139]:
tree_grid.predict(X_test) #Вот так это работает, но нам это не надо.

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [143]:
#Смотрим, какой наилучший процент правильных ответов
tree_grid.score(X_test, y_test)

0.946

In [144]:
#Лучше прямо объявлять
from sklearn.metrics import accuracy_score

In [146]:
accuracy_score(tree_grid.predict(X_test), y_test)

0.946

## Нарисуем

In [147]:
from sklearn.tree import export_graphviz

In [153]:
#Характеристики (фичи) - это и есть наши колонки здесь в начальном датафрейме
export_graphviz(tree_greed.best_estimator_, out_file='tree_3_lesson.dot', feature_names=X.columns, filled=True)

In [155]:
#Через ком строку смотрим как графвиз записал дот файл
!ls -l *.dot

-rw-rw-r-- 1 anton anton 7804 мар 23 20:56 tree_3_lesson.dot


In [156]:
export_graphviz?

In [160]:
!dot -Tpng tree_3_lesson.dot -o tree_3_lesson.png #Прямо с документации к графвизу копируем.
#Графвиз установлен через баш

In [161]:
!ls -l  #Вот и рисунок png

итого 988
-rw-rw-r-- 1 anton anton  98647 мар 23 21:02 'Oi_lesson_3_Trees&kNN.ipynb'
-rw-r--r-- 1 anton anton 310007 окт 27 19:07  telecom_churn.csv
-rw-rw-r-- 1 anton anton   6240 мар 23 20:52  tree_3_lesson
-rw-rw-r-- 1 anton anton   7804 мар 23 20:56  tree_3_lesson.dot
-rw-rw-r-- 1 anton anton 578749 мар 23 21:10  tree_3_lesson.png


В HTMLне сложно запомнить, как сюда рисунок разместить

<img src = 'tree_3_lesson.png'>